# Prepare a classification model using SVM for salary data 

Data Description:

age -- age of a person
workclass	-- A work class is a grouping of work 
education	-- Education of an individuals	
maritalstatus -- Marital status of an individulas	
occupation	 -- occupation of an individuals
relationship -- 	
race --  Race of an Individual
sex --  Gender of an Individual
capitalgain --  profit received from the sale of an investment	
capitalloss	-- A decrease in the value of a capital asset
hoursperweek -- number of hours work per week	
native -- Native of an individual
Salary -- salary of an individual


In [2]:
# SVM Classification
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler

from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder

In [3]:
filename = 'SalaryData_Test(1).csv'
SalaryTest = pd.read_csv(filename)
#filename = 'SalaryData_Train(1).csv'
#SalaryTrain = pd.read_csv(filename)

In [4]:
SalaryTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15060 entries, 0 to 15059
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            15060 non-null  int64 
 1   workclass      15060 non-null  object
 2   education      15060 non-null  object
 3   educationno    15060 non-null  int64 
 4   maritalstatus  15060 non-null  object
 5   occupation     15060 non-null  object
 6   relationship   15060 non-null  object
 7   race           15060 non-null  object
 8   sex            15060 non-null  object
 9   capitalgain    15060 non-null  int64 
 10  capitalloss    15060 non-null  int64 
 11  hoursperweek   15060 non-null  int64 
 12  native         15060 non-null  object
 13  Salary         15060 non-null  object
dtypes: int64(5), object(9)
memory usage: 1.6+ MB


In [7]:
SalaryTest

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,34,Private,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15055,33,Private,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,<=50K
15056,39,Private,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
15057,38,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
15058,44,Private,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


In [5]:
#SalaryTrain.info()

In [6]:
#to find the unique values
#SalaryTest.workclass.unique()

In [8]:
#SalaryTest:Data preprocessing : Convert All Object/Categeorical columns in numeric using Labelencoder

labelencoder = LabelEncoder()

SalaryTest["workclass"]= labelencoder.fit_transform(SalaryTest["workclass"])
SalaryTest["education"]= labelencoder.fit_transform(SalaryTest["education"])
SalaryTest["maritalstatus"]= labelencoder.fit_transform(SalaryTest["maritalstatus"])
SalaryTest["occupation"]= labelencoder.fit_transform(SalaryTest["occupation"])
SalaryTest["relationship"]= labelencoder.fit_transform(SalaryTest["relationship"])
SalaryTest["race"]= labelencoder.fit_transform(SalaryTest["race"])
SalaryTest["sex"]= labelencoder.fit_transform(SalaryTest["sex"])
SalaryTest["native"]= labelencoder.fit_transform(SalaryTest["native"])
SalaryTest["Salary"]= labelencoder.fit_transform(SalaryTest["Salary"])


In [34]:
SalaryTest

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,25,2,1,7,4,6,3,2,1,0,0,40,37,0
1,38,2,11,9,2,4,0,4,1,0,0,50,37,0
2,28,1,7,12,2,10,0,4,1,0,0,40,37,1
3,44,2,15,10,2,6,0,2,1,7688,0,40,37,1
4,34,2,0,6,4,7,1,4,1,0,0,30,37,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15055,33,2,9,13,4,9,3,4,1,0,0,40,37,0
15056,39,2,9,13,0,9,1,4,0,0,0,36,37,0
15057,38,2,9,13,2,9,0,4,1,0,0,50,37,0
15058,44,2,9,13,0,0,3,1,1,5455,0,40,37,0


In [11]:
#SalaryTest:Data preprocessing : Convert All Object/Categeorical columns in numeric using Labelencoder
"""
labelencoder = LabelEncoder()

SalaryTrain["workclass"]= labelencoder.fit_transform(SalaryTrain["workclass"])
SalaryTrain["education"]= labelencoder.fit_transform(SalaryTrain["education"])
SalaryTrain["maritalstatus"]= labelencoder.fit_transform(SalaryTrain["maritalstatus"])
SalaryTrain["occupation"]= labelencoder.fit_transform(SalaryTrain["occupation"])
SalaryTrain["relationship"]= labelencoder.fit_transform(SalaryTrain["relationship"])
SalaryTrain["race"]= labelencoder.fit_transform(SalaryTrain["race"])
SalaryTrain["sex"]= labelencoder.fit_transform(SalaryTrain["sex"])
SalaryTrain["native"]= labelencoder.fit_transform(SalaryTrain["native"])
SalaryTrain["Salary"]= labelencoder.fit_transform(SalaryTrain["Salary"])
"""

'\nlabelencoder = LabelEncoder()\n\nSalaryTrain["workclass"]= labelencoder.fit_transform(SalaryTrain["workclass"])\nSalaryTrain["education"]= labelencoder.fit_transform(SalaryTrain["education"])\nSalaryTrain["maritalstatus"]= labelencoder.fit_transform(SalaryTrain["maritalstatus"])\nSalaryTrain["occupation"]= labelencoder.fit_transform(SalaryTrain["occupation"])\nSalaryTrain["relationship"]= labelencoder.fit_transform(SalaryTrain["relationship"])\nSalaryTrain["race"]= labelencoder.fit_transform(SalaryTrain["race"])\nSalaryTrain["sex"]= labelencoder.fit_transform(SalaryTrain["sex"])\nSalaryTrain["native"]= labelencoder.fit_transform(SalaryTrain["native"])\nSalaryTrain["Salary"]= labelencoder.fit_transform(SalaryTrain["Salary"])\n'

In [12]:
#SalaryTrain

In [38]:
#SalaryTrain.shape


(30161, 14)

In [17]:
#Split X and y : SalaryTest

array = SalaryTest.values
X = array[:,0:13]
y = array[:,13]
X

array([[25,  2,  1, ...,  0, 40, 37],
       [38,  2, 11, ...,  0, 50, 37],
       [28,  1,  7, ...,  0, 40, 37],
       ...,
       [38,  2,  9, ...,  0, 50, 37],
       [44,  2,  9, ...,  0, 40, 37],
       [35,  3,  9, ...,  0, 60, 37]], dtype=int64)

In [18]:
X.shape

(15060, 13)

In [27]:
# standardization of dependent variables
scale= StandardScaler()
X = scale.fit_transform(X)
X

array([[-1.02900513, -0.222347  , -2.42343884, ..., -0.2191694 ,
        -0.07888904,  0.26058067],
       [-0.05742253, -0.222347  ,  0.19017232, ..., -0.2191694 ,
         0.75013125,  0.26058067],
       [-0.80479376, -1.25712632, -0.85527215, ..., -0.2191694 ,
        -0.07888904,  0.26058067],
       ...,
       [-0.05742253, -0.222347  , -0.33254991, ..., -0.2191694 ,
         0.75013125,  0.26058067],
       [ 0.39100021, -0.222347  , -0.33254991, ..., -0.2191694 ,
        -0.07888904,  0.26058067],
       [-0.2816339 ,  0.81243231, -0.33254991, ..., -0.2191694 ,
         1.57915155,  0.26058067]])

In [25]:
#Training model using Test Data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)

### Grid Search CV

In [26]:
clf = SVC()
param_grid = [{'kernel':['rbf'],'gamma':[50,5,10,0.5],'C':[15,14,13,12,11,10,0.1,0.001] }]
gsv = GridSearchCV(clf,param_grid,cv=10)
gsv.fit(X_train,y_train)




KeyboardInterrupt: 

In [13]:
X_train.shape

(30061, 13)

In [14]:
y_train.shape

(30061,)

In [15]:
gsv.best_params_ , gsv.best_score_ 

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [29]:
clf = SVC(C= 15, gamma = 50)
clf.fit(X_train , y_train)
y_pred = clf.predict(_X_test)
acc = accuracy_score(_y_test, y_pred) * 100
print("Accuracy =", acc)
confusion_matrix(_y_test, y_pred)

KeyboardInterrupt: 

In [ ]:
y_pred